In [228]:
import re
import csv
import time
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys

In [159]:
main_page = requests.get('https://www.rbc.ru/')
print(type(main_page))

<class 'requests.models.Response'>


In [3]:
soup = BeautifulSoup(main_page.text,'html.parser')
print(soup.prettify())

<!DOCTYPE html>
<html lang="ru">
 <head>
  <meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
  <meta content="IE=edge,chrome=1" http-equiv="X-UA-Compatible"/>
  <meta content="no-cache" http-equiv="Cache-Control"/>
  <meta content="width=device-width, initial-scale=1.0, user-scalable=no, minimum-scale=1.0, maximum-scale=1.0" name="viewport"/>
  <meta content="True" name="HandheldFriendly"/>
  <meta content="telephone=no" name="format-detection"/>
  <meta content="address=no" name="format-detection"/>
  <title>
   Последние новости дня в России и мире сегодня - свежие новости на РБК
  </title>
  <meta content="Последние новости дня в России и мире сегодня - свежие новости на РБК" name="title">
   <meta content="Новости в России и в мире о политике, экономике и бизнесе, обзоры, аналитика, оперативная информация: фото и видео с места событий на сайте РБК" name="description">
    <meta content="«РосБизнесКонсалтинг»" name="copyright">
     <meta content="index, follow" n

In [4]:
#find all the links
main_page_links = soup.find_all('a')
print(len(main_page_links))

280


In [5]:
main_page_links[0]['class']

['main-content-link', 'l-window', 'js-skip-to-content']

In [6]:
#classes that I need
classes = ['Финансы','Политика','Экономика','Бизнес','Технологии и медиа']
classes

['Финансы', 'Политика', 'Экономика', 'Бизнес', 'Технологии и медиа']

In [7]:
cls_tags = [item for item in main_page_links if item.text in classes and "js-yandex-counter" in item['class']]
cls_tags

[<a class="topline__popup__submenu-item js-yandex-counter" data-yandex-name="click_hamburger" data-yandex-params='{url: "https://www.rbc.ru/politics/?utm_source=topline"}' href="https://www.rbc.ru/politics/?utm_source=topline">Политика</a>,
 <a class="topline__popup__submenu-item js-yandex-counter" data-yandex-name="click_hamburger" data-yandex-params='{url: "https://www.rbc.ru/economics/?utm_source=topline"}' href="https://www.rbc.ru/economics/?utm_source=topline">Экономика</a>,
 <a class="topline__popup__submenu-item js-yandex-counter" data-yandex-name="click_hamburger" data-yandex-params='{url: "https://www.rbc.ru/business/?utm_source=topline"}' href="https://www.rbc.ru/business/?utm_source=topline">Бизнес</a>,
 <a class="topline__popup__submenu-item js-yandex-counter" data-yandex-name="click_hamburger" data-yandex-params='{url: "https://www.rbc.ru/technology_and_media/?utm_source=topline"}' href="https://www.rbc.ru/technology_and_media/?utm_source=topline">Технологии и медиа</a>,
 

In [8]:
dict_to_refs = dict()
for one_cls in cls_tags:
    dict_to_refs[one_cls.text] = one_cls['href']

In [9]:
dict_to_refs

{'Политика': 'https://www.rbc.ru/politics/?utm_source=topline',
 'Экономика': 'https://www.rbc.ru/economics/?utm_source=topline',
 'Бизнес': 'https://www.rbc.ru/business/?utm_source=topline',
 'Технологии и медиа': 'https://www.rbc.ru/technology_and_media/?utm_source=topline',
 'Финансы': 'https://www.rbc.ru/finances/?utm_source=topline'}

In [245]:
driver = webdriver.Chrome()

In [115]:
#function to collect links to news from a specific topic
def collect_available_news(topic_link):
    driver.get(topic_link)
    time.sleep(3)
    body = driver.find_element(By.TAG_NAME, "body")
    for _ in range(30):
        time.sleep(1.3)
        body.send_keys(Keys.END)
    text_parser = BeautifulSoup(driver.page_source,'html.parser')
    the_a_tags = text_parser.find_all('a')
    news_tags = [one_tag for one_tag in the_a_tags if 'js-rm-central-column-item-link' in one_tag['class']]
    return [one_tag['href'] for one_tag in news_tags]

In [ ]:
#function to get the training samples with targets
def get_texts_from_news(news_link,label):
    html_text = requests.get(news_link).text
    html_soup = BeautifulSoup(html_text,'html.parser')
    the_ps = html_soup.find_all('p')
    the_text_ps = [one_p.text for one_p in the_ps]
    res_texts = []
    for one_item in the_text_ps:
        if re.findall( r"РБК в Telegram",one_item):
            break
        else:
            res_texts.append((one_item,label))
    return res_texts

In [242]:
#write the news with label to a csv file
def write_to_csv(name):
    with open(name+'.csv','a',newline = '') as csvfile:
        writer = csv.writer(csvfile)
        for one_label in dict_to_refs.keys():
            all_news = collect_available_news(dict_to_refs[one_label])
            for one_news in all_news:
                time.sleep(0.8)
                all_texts = get_texts_from_news(one_news,one_label)
                for one_text in all_texts:
                    writer.writerow(one_text)

In [246]:
#starting the process of writing
write_to_csv('rbk_news')